In [1]:
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import pickle
import os
from sklearn.metrics import classification_report, accuracy_score, f1_score, plot_confusion_matrix,ConfusionMatrixDisplay, confusion_matrix
import pickle
os.environ["WANDB_DISABLED"] = "true"

In [2]:
model_name = "bert-base-uncased"
max_length=128

In [3]:
le = pickle.load(open("Data/label_encoder.pkl", "rb"))

In [4]:
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [5]:
target_names=le.transform(le.classes_)

In [6]:
def preprocess_function(examples):
    #print(examples['TEXT'])
    return tokenizer(examples["text"], padding=True, truncation=True)
    #return toknizer(examples["text"],padding='max_length',truncation=True, max_length=max_length)

In [7]:
def get_prediction(text):
    # prepare our text into tokenized sequence
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to("cuda")
    
    # perform inference to our model
    outputs = model(**inputs)
    #print(outputs)
    # get output probabilities by doing softmax
    probs = outputs[0].softmax(1)
    #print(outputs[0].softmax(1).argmax())
    # executing argmax function to get the candidate label
    return probs.argmax().item()

In [8]:
best_params = {"batch_size":8, 
              "lr": 5e-5,
              "epochs": 40}

In [9]:
train_dataset = load_dataset('csv', data_files={'train': 'Data/BERT_data_for_training'+'.csv'})
#print(train_dataset["train"][0])

tokenized_dataset_train =train_dataset.map(preprocess_function)

                                                
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#dataset has been loaded correctly. 


Using custom data configuration default-b9278eb154af2bf4
Reusing dataset csv (C:\Users\adianidm\.cache\huggingface\datasets\csv\default-b9278eb154af2bf4\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\adianidm\.cache\huggingface\datasets\csv\default-b9278eb154af2bf4\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-ac0868b7ee6ae638.arrow


In [10]:
model=BertForSequenceClassification.from_pretrained(model_name, num_labels=13)
#device="cpu"
device="cuda"
model.to(device)

#initializing training arg ith bet params
training_args = TrainingArguments(
num_train_epochs=best_params["epochs"],
learning_rate=best_params["lr"],
per_device_train_batch_size=best_params["batch_size"],  # batch size per device during training
weight_decay=0.01,               # strength of weight decay
#oad_best_model_at_end=True,
logging_steps=100,
#valuation_strategy="steps",
output_dir="Models/BERT_13classes_final_output"
)


trainer = Trainer( model=model,
              args=training_args,
              train_dataset=tokenized_dataset_train['train'],
              tokenizer=tokenizer,
              data_collator=data_collator
             )


trainer.train()

trainer.save_model("Final_Model/BERT_13classes_final")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss
100,1.539500
200,0.635800
300,0.349800
400,0.217700
500,0.118300
600,0.122800
700,0.038800
800,0.058100
900,0.050100
1000,0.042000


Saving model checkpoint to Models/BERT_13classes_final_output\checkpoint-500
Configuration saved in Models/BERT_13classes_final_output\checkpoint-500\config.json
Model weights saved in Models/BERT_13classes_final_output\checkpoint-500\pytorch_model.bin
tokenizer config file saved in Models/BERT_13classes_final_output\checkpoint-500\tokenizer_config.json
Special tokens file saved in Models/BERT_13classes_final_output\checkpoint-500\special_tokens_map.json
Saving model checkpoint to Models/BERT_13classes_final_output\checkpoint-1000
Configuration saved in Models/BERT_13classes_final_output\checkpoint-1000\config.json
Model weights saved in Models/BERT_13classes_final_output\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in Models/BERT_13classes_final_output\checkpoint-1000\tokenizer_config.json
Special tokens file saved in Models/BERT_13classes_final_output\checkpoint-1000\special_tokens_map.json
Saving model checkpoint to Models/BERT_13classes_final_output\checkpoint-1500